In [1]:
import pandas as pd; pd.set_option('precision',4)
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np; np.set_printoptions(precision=4,suppress = True)
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf, acf, pacf
import matplotlib.gridspec as gridspec
from datetime import datetime

fontsize = 15 # Set fontsize for all figures
plt.rc('font', size=fontsize)          # controls default text sizes
plt.rc('axes', titlesize=fontsize)     # fontsize of the axes title
plt.rc('axes', labelsize=fontsize)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=fontsize)    # fontsize of the tick labels
plt.rc('ytick', labelsize=fontsize)    # fontsize of the tick labels
plt.rc('legend', fontsize=fontsize)    # legend fontsize
plt.rc('figure', titlesize=fontsize)  # fontsize of the figure title


# StandardScale all x-variables - improves performance and robustness of fitted models
from sklearn.preprocessing import StandardScaler

# All subsets need their own scaler in order to call the inverse_transform method
scaler_train = StandardScaler()
scaler_test = StandardScaler()
scaler_all = StandardScaler()



In [12]:
import os 
cwd = os.getcwd()
print(os.getcwd())
os.chdir('/home/math/PycharmProjects/hmm-master')

/home/math/PycharmProjects/hmm-master


In [13]:
df = pd.read_excel('data/Adjusted Close Price Series Load.xlsx', header = 2, index_col = 'Time / Name')
df_MSCI = df[['MSCI World']]
df_MSCI.head()

,MSCI World
Time / Name,
1980-01-01,7.0217
1980-01-02,6.9258
1980-01-03,6.9075
1980-01-04,6.9630
1980-01-07,6.9889


In [14]:
df_MSCI['Returns'] = df_MSCI['MSCI World'].pct_change()
df_MSCI['Log returns'] = np.log(df_MSCI['MSCI World']) - np.log(df_MSCI['MSCI World'].shift(1))
df_MSCI.dropna(inplace=True)
df_MSCI.head(-5)

,MSCI World,Returns,Log returns
Time / Name,,,
1980-01-02,6.9258,-0.0137,-0.0138
1980-01-03,6.9075,-0.0026,-0.0027
1980-01-04,6.9630,0.0080,0.0080
1980-01-07,6.9889,0.0037,0.0037
1980-01-08,7.0757,0.0124,0.0123
...,...,...,...
2021-01-12,167.0883,0.0012,0.0012
2021-01-13,167.5403,0.0027,0.0027
2021-01-14,167.4257,-0.0007,-0.0007


### Train and Test period

In [21]:
start_train = '1980-01-01'
end_train = '2014-01-01'

start_test = '2014-01-02'
end_test = '2021-01-18'

# Get train and test as datetimes - used for plotting
start_train_dt = datetime.strptime(start_train, '%Y-%m-%d')
end_train_dt = datetime.strptime(end_train, '%Y-%m-%d')
start_test_dt = datetime.strptime(start_test, '%Y-%m-%d')
end_test_dt = datetime.strptime(end_test, '%Y-%m-%d')

df = df.loc[start_train:end_test]

In [27]:
#Get train & test sets - keep as Dataframe as oppossed to pandas series.
x_train = pd.DataFrame(df_MSCI.loc[start_train:end_train, 'Log returns'])
x_test = pd.DataFrame(df_MSCI.loc[start_test:end_test, 'Log returns'])
x_all = pd.DataFrame(df_MSCI.loc[start_train:end_test, 'Log returns'])